In [1]:
import sys
sys.path.append(r'../dotNet/libs')

import clr
clr.AddReference("Microsoft.ML.Probabilistic")
clr.AddReference("Microsoft.ML.Probabilistic.Compiler")
clr.AddReference("Microsoft.ML.Probabilistic.Learners")

from System import Array, Double, Type

In [2]:
from Microsoft.ML.Probabilistic.Models import Variable, VariableArray, Range, InferenceEngine
from Microsoft.ML.Probabilistic.Algorithms import VariationalMessagePassing
from Microsoft.ML.Probabilistic.Distributions import Gaussian, Gamma

### Example 1: coin toss

This is the implementation of https://dotnet.github.io/infer/userguide/a%20simple%20example.html

In [3]:
firstCoin = Variable.Bernoulli(Double(0.5))
secondCoin = Variable.Bernoulli(Double(0.5))
bothHeads = firstCoin.op_BitwiseAnd(firstCoin, secondCoin)

In [4]:
engine = InferenceEngine()
# This should give 'Bernoulli(0.25)' since probability
# that a coin toss will give heads for both coins equals to 0.5 * 0.5 == 0.25
print(f'Probability distribution both coins are heads: {engine.Infer(bothHeads)}');
# Now we observe a coin toss with observed value for both heads == false
bothHeads.ObservedValue = False;  
# Probability that the first coin is head == 1/3, since
# there are only 3 events possible without both coins giving heads
print(f'Probability distribution over firstCoin: {engine.Infer(firstCoin)}');

Probability distribution both coins are heads: Bernoulli(0.25)
Probability distribution over firstCoin: Bernoulli(0.3333)


### Example 2: Inference on Gaussian with priors

This is the implementation of https://dotnet.github.io/infer/userguide/Running%20inference.html

**Note**: as mentioned in the README, we can't call `Variable.Observed` as in the C# code. Another issue encountered is the usage of the `[]` indexer, which doesn't seem to be correctly resolved by python.net. The code below is a workaround that initializes the `VariableArray` differently.  

In [5]:
# define the priors for the mean and precision of the Gaussian
mean = Variable.GaussianFromMeanAndVariance(0.0, 100.0)
precision = Variable.GammaFromShapeAndScale(1.0, 1.0)

# define observed values as drawn from a Gaussian(mean, precision)
i = Range(4)
data = Variable.Array[Double](i)
data.set_Item(i, Variable.GaussianFromMeanAndPrecision(mean, precision).ForEach(i))
data.set_ObservedValue(Array[Double]([11.0, 5.0, 8.0, 9.0]))

In [6]:
# run inference
engine = InferenceEngine(VariationalMessagePassing()); 

# get the posterior distributions
posteriorMean = engine.Infer[Gaussian](mean);  
posteriorPrecision = engine.Infer[Gamma](precision);  

# This should show the following output:
# mean = Gaussian(8.165, 1.026)
# precision = Gamma(3, 0.08038)[mean=0.2411]
print(f'mean = {posteriorMean}');  
print(f'precision = {posteriorPrecision}');

mean = Gaussian(8.165, 1.026)
precision = Gamma(3, 0.08038)[mean=0.2411]
